## Exercise: Horovod with Petastorm for training a deep learning model

In this exercise we are going to build a model on the Boston housing dataset and distribute the deep learning training process using both HorovodRunner and Petastorm.

**Required Libraries**: 
* `petastorm==0.8.2` via PyPI

Run the following cell to set up our environment.

In [3]:
%run "../Includes/Classroom-Setup"

## 1. Load and process data

We again load the Boston housing data. However, as we saw in the demo, for Horovod we want to shard the data before passing into HorovodRunner. 

For the `get_dataset` function below, load the data, split into 80/20 train-test, standardize the features and return train and test sets.

In [5]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def get_dataset(rank=0, size=1):
  scaler = StandardScaler()
  
  boston_housing = load_boston()

  # split 80/20 train-test
  X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                          boston_housing.target,
                                                          test_size=0.2,
                                                          random_state=1)
  
  scaler.fit(X_train)
  X_train = scaler.transform(X_train[rank::size])
  y_train = y_train[rank::size]
  X_test = scaler.transform(X_test[rank::size])
  y_test = y_test[rank::size]
  
  return (X_train, y_train), (X_test, y_test)

##2. Build Model

Using the same model from earlier, let's define our model architecture

In [7]:
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.set_random_seed(42) # For reproducibility
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def build_model():
  return Sequential([Dense(50, input_dim=13, activation='relu'),
                    Dense(20, activation='relu'),
                    Dense(1, activation='linear')])

## 3. Horovod

In order to distribute the training of our Keras model with Horovod, we must define our `run_training_horovod` training function

In [9]:
# ANSWER
import horovod.tensorflow.keras as hvd
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import *

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  print(f"Rank is: {hvd.rank()}")
  print(f"Size is: {hvd.size()}")
  
  (X_train, y_train), (X_test, y_test) = get_dataset(hvd.rank(), hvd.size())
  
  model = build_model()
  
  from tensorflow.keras import optimizers
  optimizer = optimizers.Adam(lr=0.001*hvd.size())
  optimizer = hvd.DistributedOptimizer(optimizer)
  model.compile(optimizer=optimizer, loss="mse", metrics=["mse"])
  checkpoint_dir = f"{ml_working_path}/horovod_checkpoint_weights_lab.ckpt"
  
  callbacks = [
    # Horovod: broadcast initial variable states from rank 0 to all other processes.
    # This is necessary to ensure consistent initialization of all workers when
    # training is started with random weights or restored from a checkpoint.
    hvd.callbacks.BroadcastGlobalVariablesCallback(0),

    # Horovod: average metrics among workers at the end of every epoch.
    # Note: This callback must be in the list before the ReduceLROnPlateau,
    # TensorBoard or other metrics-based callbacks.
    hvd.callbacks.MetricAverageCallback(),

    # Horovod: using `lr = 1.0 * hvd.size()` from the very beginning leads to worse final
    # accuracy. Scale the learning rate `lr = 1.0` ---> `lr = 1.0 * hvd.size()` during
    # the first five epochs. See https://arxiv.org/abs/1706.02677 for details.
    hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
    
    # Reduce the learning rate if training plateaus.
    ReduceLROnPlateau(patience=10, verbose=1)
    
  ]
  
  # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
  if hvd.rank() == 0:
    callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))
  
  history = model.fit(X_train, y_train, callbacks=callbacks, validation_split=.2, epochs=30, batch_size=16, verbose=2)

Let's now run our model on all workers.

In [11]:
# ANSWER
from sparkdl import HorovodRunner

hr = HorovodRunner(np=0)
hr.run(run_training_horovod)

## 4. Horovod with Petastorm

We're now going to build a distributed deep learning model capable of handling data in Apache Parquet format. To do so, we can use Horovod along with Petastorm. 

First let's load the Boston housing data, and create a Spark DataFrame from the training data.

In [13]:
import pandas as pd

boston_housing = load_boston()

# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(boston_housing.data,
                                                        boston_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# concatenate our features and label, then create a Spark DataFrame from our Pandas DataFrame.
data = pd.concat([pd.DataFrame(X_train, columns=boston_housing.feature_names), 
                  pd.DataFrame(y_train, columns=["label"])], axis=1)
trainDF = spark.createDataFrame(data)
display(trainDF)

### Create Vectors

Use the VectorAssembler to combine all the features (not including the label) into a single column called `features`.

In [15]:
# ANSWER
from pyspark.ml.feature import VectorAssembler

vecAssembler = VectorAssembler(inputCols=boston_housing.feature_names, outputCol="features")
vecTrainDF = vecAssembler.transform(trainDF).select("features", "label")
display(vecTrainDF)

Let's now create a UDF to convert our Vector into an Array.

In [17]:
%scala
import org.apache.spark.ml.linalg.Vector
val toArray = udf { v: Vector => v.toArray }
spark.udf.register("toArray", toArray)

Save the DataFrame out as a parquet file to DBFS. 

Let's remember to remove the committed and started metadata files in the Parquet folder! Horovod with Petastorm will not work otherwise.

In [19]:
file_path = f"{workingDir}/petastorm.parquet"
vecTrainDF.selectExpr("toArray(features) AS features", "label").repartition(8).write.mode("overwrite").parquet(file_path)
[dbutils.fs.rm(i.path) for i in dbutils.fs.ls(file_path) if ("_committed_" in i.name) | ("_started_" in i.name)]

Let's now define our `run_training_horovod` to format our data using Petastorm and distribute the training of our Keras model using Horovod.

In [21]:
# ANSWER
from petastorm import make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset
import horovod.tensorflow.keras as hvd

abs_file_path = file_path.replace("dbfs:/", "/dbfs/")

def run_training_horovod():
  # Horovod: initialize Horovod.
  hvd.init()
  with make_batch_reader("file://" + abs_file_path, 
                         num_epochs=100, 
                         cur_shard=hvd.rank(), 
                         shard_count=hvd.size()) as reader:
    
    dataset = make_petastorm_dataset(reader).map(lambda x: (tf.reshape(x.features, [-1,13]),
                                                            tf.reshape(x.label, [-1,1])))
    
    model = build_model()
    from tensorflow.keras import optimizers
    optimizer = optimizers.Adam(lr=0.001*hvd.size())
    optimizer = hvd.DistributedOptimizer(optimizer)
    model.compile(optimizer=optimizer, loss='mse')
    
    checkpoint_dir = f"{ml_working_path}/petastorm_checkpoint_weights_lab.ckpt"
    
    callbacks = [
      hvd.callbacks.BroadcastGlobalVariablesCallback(0),
      hvd.callbacks.MetricAverageCallback(),
      hvd.callbacks.LearningRateWarmupCallback(warmup_epochs=5, verbose=1),
      ReduceLROnPlateau(monitor="loss", patience=10, verbose=1)
    ]

    # Horovod: save checkpoints only on worker 0 to prevent other workers from corrupting them.
    if hvd.rank() == 0:
      callbacks.append(ModelCheckpoint(checkpoint_dir, save_weights_only=True))

    history = model.fit(dataset, callbacks=callbacks, steps_per_epoch=10, epochs=10)

Finally, let's run our newly define Horovod training function with Petastorm to run across all workers.

In [23]:
# ANSWER
from sparkdl import HorovodRunner

hr = HorovodRunner(np=0)
hr.run(run_training_horovod)